# Download dos Arquivos

## Conjunto Teste

In [1]:
url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/105175/12693976/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1750180799&Signature=iZC%2FUZrRGa3lCrMjoRFA2A2z3FHAk4liH8mgu2fA49lsP9RyRkkuQJP1%2BIYzrPoZlKAO%2FKymLIl9YJjLjbTQJlNjSNE9KEj%2Bp5W1oPduAMMMuExewCcini2kItdsOwOcC104YJ425P%2FCQa9h3uIrrKFt%2B6bpB1RX4Xtp26Dcq6%2B4qGJ26O9ad6UThXaBF0Kwvz2kDYugaaUX3DB%2FhoebkqIs%2FpGUTgWgUoSBMO48njHNdT%2FAdydQNLU2uq571kRwrlJozglM0gYjDGmvNVTxlu0IoIxyqwH%2BGtfn2jHXvQqXZU4f36vf5MaYeVXsuUyRJ24tO6MUMi%2B7h9RCozAVnQ%3D%3D&response-content-disposition=attachment%3B+filename%3Destudo-de-caso-bcc2025-1.zip"

In [ ]:
import requests
import zipfile
import io

response = requests.get(url)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall('test')

## Conjunto Amostral

In [3]:
url = "https://drive.usercontent.google.com/download?id=1Fu369-ojLT6qHVSu-LWtyQ3BVYGTIYpd&export=download&authuser=0"

In [4]:
import requests
import io

csv_response = requests.get(url)

with open("samples.csv", "wb") as f:
    f.write(csv_response.content)

# Treinamento do Modelo

## Carregamento dos Dados

In [1]:
import pandas as pd

samples = pd.read_csv("samples.csv")

samples

,peso,altura,imc,idade,pas,pad,ppa,b2,sopro,fc,hda1,hda2,sexo,motivo1,motivo2,pc
0,25.0,110,21.0,4.25,100.0,60.0,Normal,Normal,Ausente,72.0,Assintomático,NaN,M,Parecer cardiológico,Cirurgia,Normal
1,19.3,105,18.0,3.44,100.0,60.0,Normal,Normal,Sistólico,80.0,Assintomático,NaN,F,Suspeita de cardiopatia,Sopro,Anormal
2,41.0,123,27.0,7.43,100.0,60.0,Normal,Normal,Ausente,80.0,NaN,NaN,M,Suspeita de cardiopatia,Dor precordial,Normal
3,28.0,108,24.0,4.42,90.0,60.0,Normal,Normal,Sistólico,100.0,Ganho de peso,NaN,F,Suspeita de cardiopatia,Sopro,Anormal
4,55.0,172,19.0,14.16,110.0,80.0,Normal,Normal,Sistólico,80.0,NaN,NaN,M,Suspeita de cardiopatia,Dispnéia,Anormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,51.5,153,22.0,12.86,100.0,60.0,Normal,Normal,Ausente,74.0,Assintomático,NaN,M,Parecer cardiológico,NaN,Normal
6288,42.0,142,21.0,2.79,105.0,65.0,Pre-Hipertensão PAD,Normal,Ausente,78.0,Dispneia,NaN,F,Parecer cardiológico,NaN,Normal
6289,13.9,95,15.0,3.87,90.0,65.0,Pre-Hipertensão PAD,Normal,Ausente,92.0,Assintomático,NaN,M,Parecer cardiológico,Cirurgia,Normal
6290,20.0,124,13.0,7.94,100.0,70.0,Normal,Normal,Sistólico,100.0,Assintomático,NaN,F,Parecer cardiológico,NaN,Anormal


## Pre-processamento dos Dados

In [2]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
def get_preprocessor(X):
  colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
  colunas_categoricas = X.select_dtypes(include=['object']).columns

  return make_column_transformer(
    (StandardScaler(), colunas_numericas),
    (OneHotEncoder(sparse_output=False, handle_unknown='ignore'), colunas_categoricas)
  ).fit(X)

In [4]:
samples['pc'] = samples['pc'].map({'Normal': 0, 'Anormal': 1})

X = samples.drop(columns=["pc", "sopro"])
y = samples["pc"]

In [5]:
preprocessor = get_preprocessor(X)

In [6]:
X = preprocessor.transform(X)

In [7]:
X_calib = pd.DataFrame(X).sample(frac=0.1, random_state=42)
y_calib = y[X_calib.index]

## Treinamento do Modelo

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import KFold

from scipy.stats import uniform, randint

In [9]:
estimator = RandomForestClassifier(random_state=42, n_jobs=-1)

param_distributions = {
    "criterion": [ "gini", "entropy", "log_loss"],
    "max_depth": randint(5, 100),
    "min_samples_split": randint(2, 40),
    "min_samples_leaf": randint(1, 10),
    "class_weight": [None, "balanced", "balanced_subsample" ]
}

hyperparams = HalvingRandomSearchCV(
    estimator=estimator,
    param_distributions=param_distributions,
    scoring='f1',
    factor=2,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    random_state=42,
    refit = False,
    n_jobs=-1,
    verbose=1
)

hyperparams.fit(X_calib, y_calib)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 629
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 31
n_resources: 20
Fitting 5 folds for each of 31 candidates, totalling 155 fits
----------
iter: 1
n_candidates: 16
n_resources: 40
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 8
n_resources: 80
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 4
n_resources: 160
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits


HalvingRandomSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=RandomForestClassifier(n_jobs=-1,
                                                       random_state=42),
                      factor=2, n_jobs=-1,
                      param_distributions={'class_weight': [None, 'balanced',
                                                            'balanced_subsample'],
                                           'criterion': ['gini', 'entropy',
                                                         'log_loss'],
                                           'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002267C52D370>,
                                           'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002267C6A8050>,
                                           'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002261BE11F70>},
                      random_state=42, refit=False, scoring='f1', verbose=1)

In [10]:
estimator = RandomForestClassifier(random_state=42, n_jobs=-1)
estimator.set_params(**hyperparams.best_params_)

RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_depth=95, min_samples_leaf=3, n_jobs=-1,
                       random_state=42)

In [11]:
estimator.fit(X, y)

RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_depth=95, min_samples_leaf=3, n_jobs=-1,
                       random_state=42)

## Avaliando o Modelo

In [12]:
samples_test = pd.read_csv("samples/UCMF__data_test2_kaggle.csv")

solution = pd.read_csv("samples/UCMF_solution_aleatorio_kaggle.csv")

In [13]:
X = preprocessor.transform(samples_test)
y = solution['pc'].map({'Normal': 0, 'Anormal': 1})

In [14]:
estimator.score(X, y)

0.47714285714285715

In [15]:
predict = estimator.predict_proba(X)

In [16]:
pd.DataFrame({
    'id': solution['id'],
    'pc': ['Anormal' if p[1] > 0.5 else 'Normal' for p in predict]
}).to_csv("submission.csv", index=False)

In [17]:
pd.DataFrame({
    'id': solution['id'],
    'pc': ['Anormal' if p[1] > 0.6 else 'Normal' for p in predict]
}).to_csv("submission_error.csv", index=False)

In [18]:
from sklearn.metrics import f1_score

f1_score(y, [1 if p[1] >= 0 else 0 for p in predict])

0.6666666666666666